In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.formula.api as smf
get_ipython().run_line_magic('matplotlib', 'inline')
import seaborn as sns

In [46]:
pbp = pd.read_csv("/Users/maxsteinberg/Desktop/nfl_tables/reg_pbp_2019.csv")

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (167,168,169,170,175,176,179,180,183,184,189,190,191,192,195,196,204,205,206,219,220,221,222,223,224,227,228,232,233,234,239,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
pbp.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,receiver_player_id,receiver_player_name,rusher_player_id,rusher_player_name,lateral_receiver_player_id,lateral_receiver_player_name,lateral_rusher_player_id,lateral_rusher_player_name,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,as

In [48]:
pd.set_option('display.max_columns', None)

In [49]:
pbp.columns = [i.replace(' ','_').lower() for i in pbp.columns]

In [50]:
pbp.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,receiver_player_id,receiver_player_name,rusher_player_id,rusher_player_name,lateral_receiver_player_id,lateral_receiver_player_name,lateral_rusher_player_id,lateral_rusher_player_name,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,as

In [51]:
pbp['month'] = pbp['game_date'].map(lambda x: str(x)[5:7])
pbp['day'] = pbp['game_date'].map(lambda x: str(x)[8:10])

In [52]:
import datetime
pbp['dt_date'] = pd.to_datetime(pbp['game_date'], errors='coerce')

In [53]:
pbp['day_total'] = pbp['dt_date'].dt.dayofyear

In [54]:
pbp.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,receiver_player_id,receiver_player_name,rusher_player_id,rusher_player_name,lateral_receiver_player_id,lateral_receiver_player_name,lateral_rusher_player_id,lateral_rusher_player_name,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,as

In [55]:
new_col = []
total = 253 #monday sept 10th, last game of week 1
week = 1
for _id, row in pbp.iterrows():
    if row['day_total'] - total <= 0:
        new_col.append(week)
    elif row['day_total'] - total > 0:
        new_col.append(week+1)
        total+=7
        week+=1
    else:
        new_col.append('NaN')


# In[14]:


pbp['week'] = new_col

In [56]:
pbp.home_team.value_counts()

TB     1493
CAR    1488
MIA    1485
ATL    1480
LA     1469
DET    1466
DAL    1464
ARI    1457
SF     1450
SEA    1448
PHI    1444
NYG    1441
CIN    1429
CHI    1427
CLE    1426
NYJ    1414
HOU    1414
NO     1412
IND    1405
PIT    1405
GB     1404
NE     1403
BUF    1401
TEN    1400
WAS    1397
DEN    1391
KC     1384
OAK    1382
BAL    1374
JAX    1368
MIN    1367
LAC    1358
Name: home_team, dtype: int64

In [57]:
dome_team = ['DET', 'ATL', 'IND', 'ARI', 'MIN', 'DAL', 'NO']

In [58]:
new_col = []
for _id, row in pbp.iterrows():
    if row['home_team'] in dome_team:
        new_col.append(1)
    else:
        new_col.append(0)


# In[14]:


pbp['in_dome'] = new_col

In [61]:
pbp.groupby('posteam').in_dome.sum()

posteam
ARI     752
ATL    1097
BAL       0
BUF      84
CAR     350
CHI     176
CIN       0
CLE      88
DAL     874
DEN     196
DET     880
GB      301
HOU     167
IND     771
JAX     154
KC       90
LA      289
LAC      81
MIA     171
MIN     821
NE        0
NO      760
NYG     176
NYJ       0
OAK     160
PHI     258
PIT      76
SEA     162
SF      194
TB      264
TEN     160
WAS     137
Name: in_dome, dtype: int64

In [62]:
pbp.loc[pbp['qb_scramble']==1, ['pass_attempt']] = 1

In [63]:
pbp = pbp.loc[(pbp['play_type']=='pass') | (pbp['play_type']=='run')]

In [64]:
pbp.play_type.value_counts()

pass    19063
run     13021
Name: play_type, dtype: int64

In [65]:
pbp.pass_attempt.value_counts()

1.0    19844
0.0    12240
Name: pass_attempt, dtype: int64

In [66]:
pbp.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,receiver_player_id,receiver_player_name,rusher_player_id,rusher_player_name,lateral_receiver_player_id,lateral_receiver_player_name,lateral_rusher_player_id,lateral_rusher_player_name,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,as

In [67]:
pbp = pbp[pbp['game_seconds_remaining'] >= 180]
pbp = pbp.loc[~((pbp['game_seconds_remaining']>=1800) & (pbp['game_seconds_remaining']<= 1980))]

In [68]:
pbp = pbp[pbp['yardline_100'].notnull()]
pbp = pbp[pbp['down'].notnull()]
pbp = pbp[abs(pbp['score_differential'])<=21]

In [110]:
pbp.groupby('in_dome').rfreg_pass_prob.mean()

in_dome
0    0.591323
1    0.598670
Name: rfreg_pass_prob, dtype: float64

In [86]:
pbp['score_total'] = pbp['total_home_score'] + pbp['total_away_score']

In [87]:
pbp['score_differential'].isnull().sum()

0

In [88]:
pbp = pbp[pbp['qtr']!=5]

In [89]:
from sklearn.ensemble import RandomForestClassifier

feature_cols = ['down', 'ydstogo', 'score_differential', 'qtr', 'score_total', 'in_dome']
X = pbp[feature_cols]
y = pbp.pass_attempt

In [90]:
rfreg = RandomForestClassifier(n_estimators=1000, min_samples_leaf=50, min_samples_split=100, max_features=1, oob_score=True, random_state=1)
rfreg.fit(X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=1,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=50, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=1, verbose=0,
                       warm_start=False)

In [91]:
pd.DataFrame({'feature':feature_cols, 'importance':rfreg.feature_importances_}).sort_values(by='importance')

,feature,importance
5,in_dome,0.002583
3,qtr,0.027461
4,score_total,0.048509
2,score_differential,0.124066
1,ydstogo,0.380109
0,down,0.417273


In [92]:
rfreg.oob_score_

0.6828578165526998

In [93]:
X_new = pbp[feature_cols]

In [94]:
arr = rfreg.predict_proba(X_new)
lst = []
for each in arr:
    lst.append(each[-1])
pbp['rfreg_pass_prob'] = lst

In [95]:

pbp[['down', 'ydstogo', 'score_differential', 'qtr', 'pass_attempt', 'rfreg_pass_prob', 'score_total']][pbp['down']==1].groupby('score_total').mean()



,down,ydstogo,score_differential,qtr,pass_attempt,rfreg_pass_prob
score_total,,,,,,
0,1.0,10.162701,0.000000,1.051447,0.446302,0.459119
2,1.0,9.750000,0.500000,1.000000,0.250000,0.450477
3,1.0,10.106349,-0.600000,1.344444,0.439683,0.464939
6,1.0,9.541555,-0.675603,1.630027,0.490617,0.470910
7,1.0,10.091984,-1.407359,1.266754,0.434954,0.453618
...,...,...,...,...,...,...
73,1.0,10.000000,-3.000000,4.000000,1.000000,0.623301
75,1.0,10.000000,-9.000000,4.000000,1.000000,0.683760
79,1.0,10.000000,11.000000,4.000000,0.000000,0.395155


In [96]:
expected_pass = pbp.groupby('posteam').rfreg_pass_prob.mean().to_frame(name='exp_pass').reset_index()

In [97]:
pass_perc = pbp.groupby('posteam').pass_attempt.mean().to_frame(name= 'actual_pass').reset_index()

In [98]:
total = pd.merge(expected_pass, pass_perc, on = 'posteam')

In [99]:
total['pass_above_exp'] = total['actual_pass']-total['exp_pass']

In [100]:
total.sort_values(by='pass_above_exp')

,posteam,exp_pass,actual_pass,pass_above_exp
2,BAL,0.558867,0.485313,-0.073554
19,MIN,0.592791,0.525657,-0.067133
31,WAS,0.619914,0.567529,-0.052385
13,IND,0.592698,0.542033,-0.050666
30,TEN,0.587787,0.540682,-0.047105
24,OAK,0.588556,0.543421,-0.045135
28,SF,0.571015,0.538564,-0.032451
27,SEA,0.589430,0.569544,-0.019886
26,PIT,0.587320,0.569333,-0.017986
23,NYJ,0.603605,0.586648,-0.016957


In [101]:
pbp_last4 = pbp.loc[(pbp['week']>=13) & (pbp['week']<=16)]

In [102]:
expected_pass = pbp_last4.groupby('posteam').rfreg_pass_prob.mean().to_frame(name='exp_pass').reset_index()
pass_perc = pbp_last4.groupby('posteam').pass_attempt.mean().to_frame(name= 'actual_pass').reset_index()
total_last4 = pd.merge(expected_pass, pass_perc, on = 'posteam')
total_last4['pass_above_exp'] = total_last4['actual_pass']-total_last4['exp_pass']

In [103]:
total_last4.sort_values(by='pass_above_exp')

,posteam,exp_pass,actual_pass,pass_above_exp
31,WAS,0.610542,0.510309,-0.100233
2,BAL,0.526254,0.448276,-0.077978
26,PIT,0.584956,0.513812,-0.071144
6,CIN,0.598152,0.536585,-0.061567
30,TEN,0.583486,0.528205,-0.055280
23,NYJ,0.609119,0.560976,-0.048143
20,NE,0.602319,0.554585,-0.047734
24,OAK,0.577073,0.530726,-0.046347
3,BUF,0.591094,0.552885,-0.038209
17,LAC,0.610485,0.573171,-0.037315


In [104]:
pbp_last1 = pbp[pbp['week']==1]

In [105]:
expected_pass = pbp_last1.groupby('posteam').rfreg_pass_prob.mean().to_frame(name='exp_pass').reset_index()
pass_perc = pbp_last1.groupby('posteam').pass_attempt.mean().to_frame(name= 'actual_pass').reset_index()
total_last1 = pd.merge(expected_pass, pass_perc, on = 'posteam')
total_last1['pass_above_exp'] = total_last1['actual_pass']-total_last1['exp_pass']

In [106]:
total_last1.sort_values(by='pass_above_exp')

,posteam,exp_pass,actual_pass,pass_above_exp
19,MIN,0.534222,0.323529,-0.210692
13,IND,0.613862,0.480000,-0.133862
27,SEA,0.614387,0.486486,-0.127900
2,BAL,0.495297,0.368421,-0.126876
28,SF,0.577829,0.469388,-0.108442
9,DEN,0.628677,0.555556,-0.073122
10,DET,0.558355,0.517857,-0.040498
29,TB,0.620028,0.584906,-0.035122
24,OAK,0.559666,0.530612,-0.029054
25,PHI,0.579803,0.560606,-0.019197
